# Imports

In [40]:
# import lux
import pandas as pd
import numpy as np

import db_func

try:
    %load_ext autotime
except:
    !pip3 install ipython-autotime
    %load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 497 µs (started: 2021-07-22 16:59:23 -04:00)


## Get Database Connection

In [4]:
conn = db_func.get_conn()

time: 3.81 ms (started: 2021-07-22 15:45:40 -04:00)


# Preprocessing

## Populate Dataframes From Database

In [28]:
match_query = '''SELECT *
            FROM match
            ORDER BY date ASC
            '''
season_query = '''SELECT *
            FROM season
            '''
match_df = pd.read_sql(match_query, conn)
match_df.sort_values('date', inplace=True)

season_df = pd.read_sql(season_query, conn)

time: 34.8 ms (started: 2021-07-22 16:28:17 -04:00)


## Season

Set the season for each match

In [55]:
def get_season(date):
    return season_df[(season_df['start_date'] <= date) &
                     (season_df['end_date'] >= date)]['season']

15    2021
Name: season, dtype: int64

time: 2.16 ms (started: 2021-07-22 17:15:30 -04:00)


In [ ]:
match_df['season'] = match_df['date'].map(get_season)

# Feature Engineering

## Team Elo

In [76]:
def calculate_team_elo(df):
    df["home_elo"] = df["home_elo"] + 1
    return


def get_prev_match(date, team, match_df):
    return match_df[(match_df["date"] < date) &
                    ((match_df["home_abbr"] == team) |
                     (match_df["away_abbr"] == team))]


def get_prev_elo(match, prev_match):
    if match['team'] == prev_match['home_abbr'].values[0]:
        elo = prev_match['home_elo'].values[0]
    else:
        elo = prev_match['away_elo'].values[0]

    if prev_match['season'] != match['season']:
        elo = elo * 0.75 + 1505 * 0.25
    return elo

Each team starts at 1500 elo

In [58]:
match_df['home_elo'] = 1500
match_df['away_elo'] = 1500

time: 1.09 ms (started: 2021-07-22 17:33:13 -04:00)


In [81]:
match_df.iloc[1:5].apply(lambda x: print(x['home_abbr']), axis=1)

SAS
PHO
PHI
BOS


1    None
2    None
3    None
4    None
dtype: object

In [80]:
match_df.loc[1:, 'home_elo'] = match_df \
    .apply(lambda x: get_prev_elo(x, get_prev_match(x['date'],
                                                    x['home_abbr'],
                                                    match_df)), axis=1)

KeyError: 'team'

In [62]:
%unload_ext autotime